In [30]:
import pandas as pd
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm')
from tqdm.notebook import tqdm 

In [144]:
df = pd.read_csv('00154dd0-3c43-4c79-929b-daf93c654489.csv')

In [148]:
df = df.drop(columns = 'filename').dropna(axis='rows')

In [149]:
df

,hl1,author,lede,body,DOC-ID,pubDay,pubMonth,pubYear,pubName
0,Top chefs bring the heat,SOCIAL STUDIES by ERICA CORSANO,It may not have been as intense as 'Hell's Kit...,"Chefs Jason Bond of Bondir, Bill Brodsky of Bo...",BHERLD-25225876,17,3,2015,The Boston Herald
1,Wilfork goes out a legend,JEFF HOWE,Vince Wilfork will be immortalized as a Patrio...,'Few players reached or will ever reach the sp...,BHERLD-25185125,17,3,2015,The Boston Herald
2,Hillary's modus operandi not pretty,JENNIFER C. BRACERAS,Just when you thought we were done with emailg...,Didn't Clinton tell us that she set up the pri...,BHERLD-25223487,17,3,2015,The Boston Herald
3,Coen brings NU up to speed on ND,TOM LAYMAN,Pulling Bill Coen away from basketball is a to...,Coen picked the right team to view. The 14th-...,BHERLD-25185151,17,3,2015,The Boston Herald
5,Elgort runs toward new challenge with 'Insurgent',STEPHEN SCHAEFER / MOVIES,LOS ANGELES - A year after his breakthrough su...,"'It's a big franchise so that's nice, and it's...",BHERLD-25187703,17,3,2015,The Boston Herald
6,RETAILERS FEAR TRASHED STREETS,MARIE SZANISZLO,With dirty snowbanks still lining many streets...,'After this very tough winter for main streets...,BHERLD-25226117,17,3,2015,The Boston Herald
7,Night of the laughing dead,MARK A. PERIGARD,It's the 'Walking Dead' 'Quincy'/'Psych' mash...,"One night, the super-overachiever decides to t...",BHERLD-25223152,17,3,2015,The Boston Herald
8,Sox see what's missing,JASON MASTRODONATO,"FORT MYERS - It's not just the results, the co...","The Red Sox don't have a Matt Harvey, and they...",BHERLD-25185113,17,3,2015,The Boston Herald
9,Spandex-loving speeders want to be king of road,HOWIE CARR,Just what we need here in Massachusetts - anot...,"I thought they already were! These sweaty, tru...",BHERLD-25225806,17,3,2015,The Boston Herald
10,Try to find method for Madness,TOM LAYMAN,There's no better feeling than filling out a b...,There is no foolproof way to pick a bracket. S...,BHERLD-25185150,17,3,2015,The Boston Herald


In [140]:
def combine_led_body(df):
    body = df['body'].values
    lede = df['lede'].values
    arr = np.vstack((lede,body)).T
    for step, x in enumerate(arr): 
        arr[step] = ' '.join(x)

    arr = arr.flatten()
    return arr 

In [150]:
corpus = combine_led_body(df)

In [88]:
def custom_preprocess(corpus_iterable): 
    return [[z.lemma_.lower() for z in y if not (z.is_stop or z.text == '\n')] for y in 
tqdm(nlp.pipe(corpus_iterable), total = len(corpus_iterable)) ]

In [152]:
pre_processed_corpus = custom_preprocess(corpus)

  0%|          | 0/74 [00:00<?, ?it/s]

In [153]:
pre_processed_corpus

[['intense',
  "'",
  'hell',
  'kitchen',
  ',',
  "'",
  'compete',
  'chef',
  'yesterday',
  'greater',
  'boston',
  'culinary',
  'cook',
  '-',
  'certainly',
  ' ',
  'bring',
  'game',
  ',',
  'heat',
  'thing',
  'thenew',
  'england',
  'food',
  'main',
  'stage',
  '.',
  ' ',
  'chefs',
  'jason',
  'bond',
  'bondir',
  ',',
  'bill',
  'brodsky',
  'boston',
  'nightlife',
  'ventures',
  'david',
  'ladner',
  'rialto',
  'give',
  'local',
  'scallop',
  'new',
  'zealand',
  'lamb',
  'main',
  'ingredient',
  'create',
  'appetizer',
  'entree',
  '.',
  ' ',
  'judges',
  'scott',
  'kearnan',
  '(',
  'zagat',
  ')',
  ',',
  'henry',
  'santoro',
  '(',
  'radio',
  'bdc',
  ')',
  'task',
  'award',
  'point',
  'taste',
  ',',
  'presentation',
  'creativity',
  ',',
  'gorgeous',
  'emcee',
  'jenny',
  'johnson',
  'chat',
  ',',
  'audience',
  'chef',
  'signature',
  ',',
  'fun',
  '-',
  'love',
  'banter',
  '.',
  'student',
  'le',
  'cordon',
  'ble